In [1]:
import pandas as pd
import os

In [2]:
current_dir = os.getcwd() #current dir
parent_dir = os.path.dirname(current_dir) #parent of current dir

In [3]:
movies = pd.read_csv(os.path.join(parent_dir,'raw_data/movies.csv'))
actors = pd.read_csv(os.path.join(parent_dir,'raw_data/actors.csv'))
crew = pd.read_csv(os.path.join(parent_dir,'raw_data/crew.csv'))
languages = pd.read_csv(os.path.join(parent_dir,'raw_data/languages.csv'))
genres = pd.read_csv(os.path.join(parent_dir,'raw_data/genres.csv'))
studios = pd.read_csv(os.path.join(parent_dir,'raw_data/studios.csv'))
countries = pd.read_csv(os.path.join(parent_dir,'raw_data/countries.csv'))

In [4]:
movies = movies.drop(columns='tagline', axis=1)

actors = actors.drop(columns='role', axis=1)
actors = actors.dropna()

crew = crew[crew['role'].isin(['Director', 'Writer', 'Cinematography', 'Composer'])] #'Songs', 'Producer',

languages = languages[languages['type'].isin(['Language', 'Primary language'])].drop(columns='type')

In [5]:
new_crew = (
    crew.groupby('id')
    .apply(lambda x: x.groupby('role')['name'].apply(list).to_dict())
    .reset_index(name='crew_dict')
)

/tmp/ipykernel_23619/2434692008.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.groupby('role')['name'].apply(list).to_dict())


In [6]:
new_genres = (
    genres.groupby('id')['genre']
    .apply(list)  # Aggregates genres into a list
    .reset_index(name='genre_list')  # Converts to DataFrame and renames the column
)

new_studios = (
    studios.groupby('id')['studio']
    .apply(list)  # Aggregates genres into a list
    .reset_index(name='studio_list')  # Converts to DataFrame and renames the column
)

new_actors = (
    actors.groupby('id')['name']
    .apply(list)  # Aggregates genres into a list
    .reset_index(name='actor_list')  # Converts to DataFrame and renames the column
)

In [1]:
# actors = pd.read_csv('raw_data/actors.csv')
# crew = pd.read_csv('raw_data/crew.csv')
# languages = pd.read_csv('raw_data/languages.csv')
# genres = pd.read_csv('raw_data/genres.csv')
# studios = pd.read_csv('raw_data/studios.csv')
# countries = pd.read_csv('raw_data/countries.csv')

# temp = crew[crew['id']==1000001]
#consider relabel those with small proportion to new category "Others"
#embeddings make sense, but we will have little control
#TODO: we check later how we deal with this feature!

# crew.isnull().sum()
# crew.role.unique()

# genres.genre.unique() #OneHotEncoder

# data.date.isnull().count()

# studios.studio.value_counts()/studios.studio.value_counts().sum()
#consider relabel those with small proportion to new category "Others" or drop them
#embeddings make sense, but we will have little control
#TODO: we check later how we deal with this feature!

# countries # categorical, high-cardinality, embeddings
# countries # if we chose studios, this one can be left out due to collinearity

# Incremental Recall
# python surprise
# Qualitative not

# - dropna, fillna: imputer
# - solve outliners (year range): drop those that have

In [7]:
data = movies \
    .merge(new_genres, how='left', on='id') \
    .merge(new_actors, how='left', on='id') \
    .merge(languages, how='left', on='id') \
    .merge(new_studios, how='left', on='id') \
    .merge(new_crew, how='left', on='id')

# data.to_csv(os.path.join(parent_dir,'compiled_movies.csv'), index=False)

In [46]:
data.to_csv(os.path.join(parent_dir,'compiled_movies.csv'), index=False)

In [8]:
# CLEANUP A
data = data.dropna(subset=['date'])
data['date'] = data['date'].astype(int)
data = data.loc[data['name'] != 'Untitled'] #to be completed
data = data.loc[data['date'] <= 2024]

In [9]:
data['key_b'] = data['name'] + data['date'].apply(
    lambda x: f" ({int(x)})" if not pd.isna(x) else ''
    )

In [11]:
check_data = data.groupby('key_b').size().reset_index(name='frequency').sort_values(by='frequency', ascending=False)

In [12]:
unique_keys = data['key_b'].value_counts()  # Count occurrences
unique_keys = unique_keys[unique_keys == 1].index  # Keep only keys with frequency 1
data = data[data['key_b'].isin(unique_keys)] # Filter rows where key_b is unique

In [13]:
del unique_keys

In [14]:
# data.info()
# 941597 entries original, after dropping key_b dup then 900394 entries, after dropping those that have no year 817987, after dropping those that have date >2024 817572

In [37]:
data.head()

,id,name,date,description,minute,rating,genre_list,actor_list,language,studio_list,crew_dict,key_b
1,1000002,Parasite,2019,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56,"[Comedy, Thriller, Drama]","[Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Ch...",Korean,[Barunson E&A],"{'Cinematography': ['Hong Kyung-pyo'], 'Compos...",Parasite (2019)
2,1000003,Everything Everywhere All at Once,2022,An aging Chinese immigrant is swept up in an i...,140.0,4.30,"[Science Fiction, Adventure, Comedy, Action]","[Michelle Yeoh, Ke Huy Quan, Stephanie Hsu, Ja...",English,"[IAC Films, AGBO, Ley Line Entertainment, Year...","{'Cinematography': ['Larkin Seiple'], 'Compose...",Everything Everywhere All at Once (2022)
3,1000004,Fight Club,1999,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27,[Drama],"[Edward Norton, Brad Pitt, Helena Bonham Carte...",English,"[Fox 2000 Pictures, Regency Enterprises, The L...","{'Cinematography': ['Jeff Cronenweth'], 'Compo...",Fight Club (1999)
4,1000005,La La Land,2016,"Mia, an aspiring actress, serves lattes to mov...",129.0,4.09,"[Drama, Comedy, Music, Romance]","[Ryan Gosling, Emma Stone, John Legend, Rosema...",English,"[Summit Entertainment, Black Label Media, Gilb...","{'Cinematography': ['Linus Sandgren'], 'Compos...",La La Land (2016)
5,1000006,Oppenheimer,2023,The story of J. Robert Oppenheimer's role in t...,181.0,4.23,"[Drama, History]","[Cillian Murphy, Emily Blunt, Matt Damon, Robe...",English,"[Syncopy, Universal Pictures, Atlas Entertainm...","{'Cinematography': ['Hoyte van Hoytema'], 'Com...",Oppenheimer (2023)


In [4]:
data = pd.read_csv(os.path.join(parent_dir,'compiled_movies.csv'))

In [5]:
clean_up_data = data.copy()

In [6]:
clean_up_data = clean_up_data.dropna(subset='crew_dict')

In [7]:
clean_up_data = clean_up_data.drop(columns=['studio_list', 'rating'])

In [ ]:
clean_up_data.minute #impute mean or median
clean_up_data.genre_list #impute with new genre category "Other" or "Unknown"
clean_up_data.actor_list #drop rows

# SUPER SOLUTION: API scrapping can fill in all missing values
# some fancy graphs in the frontend if possible

In [8]:
clean_up_data = clean_up_data.dropna()

In [9]:
clean_up_data.isnull().sum()

id             0
name           0
date           0
description    0
minute         0
genre_list     0
actor_list     0
language       0
crew_dict      0
key_b          0
dtype: int64

In [12]:
clean_up_data.to_csv(os.path.join(parent_dir,'clean_compile.csv'))

In [ ]:
# ADVISE FROM TEACHERS
# first drop rows where everything is missing
# for Wed: drop rows where description isnull, no impute using other text columns --> future: API web_scrapping to fill in description
# for Wed: snapshot dataset (random/first/last 10k)

In [14]:
clean_up_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 379203 entries, 0 to 817568
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           379203 non-null  int64  
 1   name         379203 non-null  object 
 2   date         379203 non-null  int64  
 3   description  379203 non-null  object 
 4   minute       379203 non-null  float64
 5   genre_list   379203 non-null  object 
 6   actor_list   379203 non-null  object 
 7   language     379203 non-null  object 
 8   crew_dict    379203 non-null  object 
 9   key_b        379203 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 31.8+ MB


In [ ]:
number
category
text

In [9]:
clean_up_data.shape

(721472, 10)

In [49]:
check_series = data.loc[data.crew_dict.isnull()]

In [45]:
133223/817572 * 100 # % of description isnull

# CREW
# 1. drop crew_dict isnull

# DESCRIPTION
# 1. drop rows where description isnull --
# 2. impute description with combi of genre, language, etc. --Aybike, Ha, []
# 3. impute description with random text

16.294956285195678

In [15]:
new_A = data[['key_b', 'date']]

In [16]:
new_A

,key_b,date
1,Parasite (2019),2019
2,Everything Everywhere All at Once (2022),2022
3,Fight Club (1999),1999
4,La La Land (2016),2016
5,Oppenheimer (2023),2023
...,...,...
940966,A Nursery In Paris (1909),1909
940967,I klädloge och på scen (1908),1908
940968,Danse fantaisiste (1902),1902
940969,La Gigue (1902),1902


## Process dataset B

In [17]:
B_movie = pd.read_csv(os.path.join(parent_dir,'raw_data/Set B/movie.csv'))
B_rating = pd.read_csv(os.path.join(parent_dir,'raw_data/Set B/rating.csv'))

B_data = B_movie.merge(B_rating, how='left', on='movieId')

In [18]:
B_data['year'] = B_data['title'].str.extract(r'\((\d{4})\)')
B_data['new_title'] = B_data['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)
B_data['new_title'] = B_data['new_title'].apply(
    lambda x: f"The {x[:-5]}" if x.endswith(', The') else x
)

In [26]:
# new_B = B_data[['new_title']].drop_duplicates()
# Modify titles ending with ', The'

new_B = B_data[['title', 'new_title', 'year']].drop_duplicates()

In [30]:
new_B['key_a'] = B_data['new_title'] + B_data['year'].apply(
    lambda x: f" ({int(x)})" if not pd.isna(x) else ''
    )

In [31]:
new_B.head()

,title,new_title,year,key_a
0,Toy Story (1995),Toy Story,1995,Toy Story (1995)
49695,Jumanji (1995),Jumanji,1995,Jumanji (1995)
71938,Grumpier Old Men (1995),Grumpier Old Men,1995,Grumpier Old Men (1995)
84673,Waiting to Exhale (1995),Waiting to Exhale,1995,Waiting to Exhale (1995)
87429,Father of the Bride Part II (1995),Father of the Bride Part II,1995,Father of the Bride Part II (1995)


## Join A and B

In [32]:
merge_A_and_B = new_A.merge(new_B, how='left', left_on='key_b', right_on='key_a')

In [33]:
merge_A_and_B[merge_A_and_B['title'].notna()]

,key_b,date,title,new_title,year,key_a
2,Fight Club (1999),1999,Fight Club (1999),Fight Club,1999,Fight Club (1999)
5,Interstellar (2014),2014,Interstellar (2014),Interstellar,2014,Interstellar (2014)
8,Pulp Fiction (1994),1994,Pulp Fiction (1994),Pulp Fiction,1994,Pulp Fiction (1994)
10,Whiplash (2014),2014,Whiplash (2014),Whiplash,2014,Whiplash (2014)
15,The Dark Knight (2008),2008,"Dark Knight, The (2008)",The Dark Knight,2008,The Dark Knight (2008)
...,...,...,...,...,...,...
719244,Die Frau des Frisörs (2008),2008,Die Frau des Frisörs (2008),Die Frau des Frisörs,2008,Die Frau des Frisörs (2008)
735576,Confessions of a Gangsta (2006),2006,Confessions of a Gangsta (2006),Confessions of a Gangsta,2006,Confessions of a Gangsta (2006)
752416,Apollo Zero (2009),2009,Apollo Zero (2009),Apollo Zero,2009,Apollo Zero (2009)
764628,Family Meeting (2007),2007,Family Meeting (2007),Family Meeting,2007,Family Meeting (2007)


# SUMMARY
- import csv files
- 

ADVISE: high-cardiality --> may make sense to drop the rare guys or not